In [4]:
import mysql.connector
import configparser

In [5]:
# Leer las credenciales del archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

user = config['database']['user']
password = config['database']['password']
host = config['database']['host']
database = config['database']['database']

In [6]:
# Abrir la base de datos: establecer la conexión
cnx = mysql.connector.connect(user=user, password=password, host=host, database=database)

# crear un cursor
cursor = cnx.cursor()

# seleccionar la base de datos
cursor.execute("USE flujo_caja_directo ")

# =====================================================================================================================

# crear tabla de agregación "tbl_saldo_mes" 
cursor.execute("""
    CREATE TABLE tbl_saldo_mes (
    id_saldo_mes INT(11) NOT NULL AUTO_INCREMENT,
    fecha DATE,
    mes INT,
    anio INT,
    saldo_inicial DECIMAL(18,2),
    neto_mes DECIMAL(18,2),
    saldo_final DECIMAL(18,2),
    PRIMARY KEY (id_saldo_mes)
    );
""")

# =====================================================================================================================

# En MySQL, el primer día de la semana es el domingo, el último es el sábado.
# En MySQL, cuando el año comienza en día diferencte a domingo, la primera semana del año es la semana 0
# Por lo tanto, la semana 1 del año es la primera semana que tiene al menos 4 días en el año    
# Para obtener el número de semana en MySQL se utiliza la función WEEK(fecha, 1)

# crear tabla de agregación "tbl_saldo_semana" 
cursor.execute("""
    CREATE TABLE tbl_saldo_semana (
    id_saldo_semana INT(11) NOT NULL AUTO_INCREMENT,
    fecha DATE,
    anio INT,
    mes INT,
    semana INT,
    saldo_inicial DECIMAL(18,2),
    neto_mes DECIMAL(18,2),
    saldo_final DECIMAL(18,2),
    PRIMARY KEY (id_saldo_semana)
    );
""")

# No se requieren llaves foraneas porque no se harán registros manuales en esta tabla
# Es decir, esta tabla únicamente se llenará con los datos de la tabla de transacciones
# Si se requiere un registro manual, se deberá hacer en el informe de flujo de caja que se genere en excel
# =====================================================================================================================

# Bajo el diseño de tablas y modelo para resumir y actualizar → Ya no se necesitan los triggres!!!!

# Cerrar cursor
cursor.close()

# cerrar la conexión
cnx.commit()
cnx.close()